In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import random
from feret_utils import get_feret_files_and_tags_dict, run_face_detection, create_dataset, create_dataset_gs
from matplotlib.pyplot import imshow
from IPython.display import Image
import cv2 as cv
import tensorflow as tf

In [2]:
# Start by getting a dictionary holding file names of images as keys and id of person as value
feret_dict = get_feret_files_and_tags_dict()

In [3]:
# During training we want to see the affect of not using the "pr" and "pl" examples
list_of_excluded = []

In [5]:
# Create a datast, using only images which were detected in run_face_detection run, and crop the images to new size
subject_list, data_dict, mean_image, std_image = create_dataset_gs(feret_dict, subject_count=700, flip=True)

In [8]:
import pickle
# Save data_dict so we can unpickle it when needed
data_to_pickle = (subject_list, data_dict, mean_image, std_image)
with open('feret_data_dict_gs.pickle', 'wb') as handle:
    pickle.dump(data_to_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
import pickle
# Unpickle data_dict
with open('feret_data_dict_gs.pickle', 'rb') as handle:
    subject_list, data_dict, mean_image, std_image = pickle.load(handle)

In [6]:
# List of images to exclude after manually going over dataset and deleting irrelevant pictures
ex_list = [119, 120, 167, 168, 193, 194, 253, 254, 341, 342, 515, 516, 547, 548, 669, 670, 775, 776, 1171, 1172,
 1181, 1182, 1213, 1214, 1275, 1276, 1395, 1396, 1517, 1518, 1521, 1522, 1531, 1532, 1613, 1614, 1685, 1686,
 1695, 1696, 1701, 1702, 1703, 1704, 1725, 1726, 1747, 1748, 1779, 1780, 1789, 1790, 1899, 1900, 1937, 1938,
 1945, 1946, 1951, 1952, 1953, 1954, 1955, 1956, 1987, 1988, 2093, 2094, 2107, 2108, 2115, 2116, 2125, 2126,
 2163, 2164, 2181, 2182, 2199, 2200, 2213, 2214, 2229, 2230, 2255, 2256, 2281, 2281, 2283, 2284, 2311, 2312,
 2333, 2334, 2381, 2382, 2397, 2398, 2411, 2412, 2497, 2498, 2519, 2520, 2673, 2674, 2683, 2684, 2743, 2744,
 2761, 2762, 2777, 2778, 2833, 2834, 2857, 2858, 2873, 2874, 2875, 2876, 2919, 2920, 2939, 2940, 3021, 3022,
 3067, 3068, 3105, 3106, 3143, 3144, 3183, 3184, 3213, 3214, 3227, 3228, 3233, 3234, 3245, 3246, 3251, 3252,
 3252, 3254, 3267, 3268, 3353, 3354, 3401, 3402, 3405, 3406, 3407, 3408, 3417, 3418, 3447, 3448, 3717, 3718,
 3741, 3742, 3759, 3760, 3813, 3814, 3863, 3864, 3865, 3866, 3871, 3872, 3875, 3876, 3883, 3884, 3913, 3914,
 3943, 3944, 3963, 3964, 3967, 3968, 3973, 3981, 3982, 3999, 4000, 4013, 4014, 4095, 4096, 4113, 4114, 4141,
 4142, 4159, 4160, 4175, 4176, 4181, 4182, 4183, 4184, 4195, 4196, 4219, 4220, 4223, 4224, 4233, 4234, 4261,
 4262, 4263, 4264, 4281, 4282, 4319, 4320, 4349, 4350, 4351, 4352, 4371, 4372, 4377, 4378, 4397, 4398, 4405,
 4406, 4407, 4408, 4421, 4422, 4425, 4426, 4455, 4456, 4577, 4578, 4631, 4632, 4641, 4642, 4645, 4646, 4657,
 4658, 4667, 4668, 4685, 4686 ,4757, 4758, 4787, 4788, 4805, 4806, 4881, 4882, 4897, 4898, 4905, 4906, 5009,
 5010, 5021, 5022, 5049, 5050, 5059, 5060, 5087, 5088, 5139, 5140, 5147, 5148, 5237, 5238, 5247, 5248, 5261,
 5262, 5273, 5274, 5283, 5284, 5297, 5298, 5353, 5354, 5355, 5356, 5379, 5380, 5405, 5406, 5429, 5430, 5431,
 5432, 5551, 5552, 5561, 5562, 5649, 5650, 5653, 5654, 5675, 5676, 5687, 5688, 5703, 5704, 5729, 5730, 5735,
 5736, 5741, 5742, 5749, 5750, 5819, 5820, 5823, 5824, 5833, 5834, 5895, 5896, 5945, 5946, 5985, 5986, 6077,
 6078, 6123, 6124, 6141, 6142, 6221, 6222, 6321, 6322, 6341, 6342, 6351, 6352, 6371, 6372, 6433, 6434, 6531,
 6532, 6535, 6536, 6545, 6546, 6553, 6554, 6677, 6678, 6679, 6680, 6681, 6682, 6683, 6684, 6693, 6694, 6759,
 6760, 6789, 6790, 6807, 6808, 6873, 6874, 6875, 6876, 6893, 6894, 6909, 6910, 6939, 6940, 6993, 6994, 7029,
 7030, 7037, 7038, 7087, 7088, 7103, 7104, 7121, 7122, 7123, 7124, 7129, 7130, 7251, 7252, 7271, 7272, 7277,
 7278, 7281, 7282]


for i in reversed(ex_list):
    data_dict['X_train'] = np.delete(data_dict['X_train'], i-1, axis=0)
    data_dict['y_train'] = np.delete(data_dict['y_train'], i-1, axis=0)

In [14]:
print(data_dict['X_train'].shape)
print(data_dict['y_train'].shape)
print(data_dict['X_eval'].shape)
print(data_dict['y_eval'].shape)

(6931, 96, 96, 1)
(6931,)
(268, 96, 96, 1)
(268,)


In [15]:
tf.logging.set_verbosity(tf.logging.ERROR)
tf.reset_default_graph()

In [16]:
num_of_classes = len(subject_list)
def cnn_model_fn(features, labels, mode):
    """CNN model function"""
    
    input_layer = tf.reshape(features["x"], [-1, 96, 96, 1])
    conv1 = tf.layers.conv2d(inputs=input_layer,
                            filters=32,
                            kernel_size=7,
                            padding="same",
                            kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
                            activation=tf.nn.leaky_relu)
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=2, strides =2)
    conv2 = tf.layers.conv2d(inputs=pool1,
                            filters=32,
                            kernel_size=5,
                            padding="same",
                            kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
                            activation=tf.nn.leaky_relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv1, pool_size=2, strides =2)
    conv3 = tf.layers.conv2d(inputs=pool2,
                            filters=32,
                            kernel_size=3,
                            padding="same",
                            kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
                            activation=tf.nn.leaky_relu)
    pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=2, strides =2)
    pool3_flat = tf.reshape(pool3, [-1, 24 * 24 * 32])
    dense = tf.layers.dense(inputs=pool3_flat, units=24 * 24 * 32, activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense, rate=0.1, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    logits = tf.layers.dense(inputs=dropout, units=num_of_classes)
    classes = tf.as_string(tf.argmax(input=logits, axis=1, name="class"))
    predictions = {
        "classes" : tf.argmax(input=logits, axis=1),
        "probabilities" : tf.nn.softmax(logits, name="softmax_tensor"),
        "dense" : dense,
        "pool3" : pool3_flat
        }
        
    if mode == tf.estimator.ModeKeys.PREDICT:
        export_outputs = {"predictions" : tf.estimator.export.PredictOutput(predictions)}
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions, export_outputs=export_outputs)
    
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth = num_of_classes)
    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        # This is wrong, need to fix
        metrics = {
            "accuracy" : tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])
        }
        tf.summary.scalar("accuracy", metrics["accuracy"][1])
        merge_summary_op = tf.summary.merge_all
        lr = 1e-4
        step_rate = 10000
        decay = 0.8
        learning_rate = tf.train.exponential_decay(lr, global_step=tf.train.get_or_create_global_step(), decay_steps=step_rate, decay_rate=decay, staircase=True)
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    eval_metrics_op = {"accuracy" : tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
    
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metrics_op)

In [19]:
face_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="train/train_gs_0/")
tensors_to_log = {"probabilities" : "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=200)

In [20]:
training_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x" : data_dict['X_train']},
                                                       y=data_dict['y_train'],
                                                       batch_size=25,
                                                       num_epochs=None,
                                                       shuffle=True)
face_classifier.train(input_fn=training_input_fn, steps=1000, hooks=[logging_hook])

In [21]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x" : data_dict['X_eval']},
                                                   y=data_dict['y_eval'],
                                                   num_epochs=1,
                                                   shuffle=False)
eval_results = face_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

{'accuracy': 0.6268657, 'loss': 1.5606836, 'global_step': 1000}


In [22]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x" : data_dict['X_train']},
                                                   y=data_dict['y_train'],
                                                   num_epochs=1,
                                                   shuffle=False)
eval_results = face_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

{'accuracy': 0.95729333, 'loss': 0.17117465, 'global_step': 1000}


In [23]:
predict_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x" : data_dict['X_train'][:,:,:,:]}, num_epochs=1, shuffle = False)
predictions = list(face_classifier.predict(input_fn=predict_input_fn))
print(len(predictions))

6931


In [18]:
with open('known_faces/faces.pickle', 'rb') as handle:
    known_pics_dict = pickle.load(handle)

dense_dict = {}
    
for file, pic in known_pics_dict.items():
    pic = (pic - mean_image)/std_image
    predict_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x" : pic}, num_epochs=1, shuffle = False)
    prediction = list(face_classifier.predict(input_fn=predict_input_fn))[0]
    
    dense_dict[file] = prediction["dense"]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train/train_gs_2/model.ckpt-14476
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train/train_gs_2/model.ckpt-14476
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train/train_gs_2/model.ckpt-14476
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train/train_gs_2/model.ckpt-14476
INFO:te

In [19]:
for file, dense in dense_dict.items():
    lowest = 1000
    key_file = ''
    for key in dense_dict.keys():
        if key == file:
            continue
        curr = np.mean((dense - dense_dict[key])**2)
        if curr < lowest:
            lowest = curr
            key_file = key
    print("Original is {} and closest is {} with score of {}".format(file, key_file, lowest))

Original is shahar4.jpg and closest is michi2.jpg with score of 0.24735122919082642
Original is michi4.jpg and closest is shahar4.jpg with score of 0.5207279324531555
Original is shahar1.jpg and closest is shahar2.jpg with score of 0.6190715432167053
Original is shahar2.jpg and closest is shahar5.jpg with score of 0.42375075817108154
Original is michi5.jpg and closest is michi3.jpg with score of 0.6107949614524841
Original is michi2.jpg and closest is michi3.jpg with score of 0.15404987335205078
Original is shahar5.jpg and closest is shahar2.jpg with score of 0.42375075817108154
Original is michi3.jpg and closest is michi2.jpg with score of 0.15404987335205078


In [9]:
def serving_input_receiver_fn():
    """An input receiver that expects a serialized tf.Example."""
    serialized_tf_example = tf.placeholder(dtype=tf.string, shape=[None], name='input_tensors')
    receiver_tensors = {"predictor_inputs" : serialized_tf_example}
    feature_spec = {"x" : tf.FixedLenFeature([96, 96 ,1], tf.float32)}
    features = tf.parse_example(serialized_tf_example, feature_spec)
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensors)

exported_model = face_classifier.export_savedmodel("trained/trained_0", serving_input_receiver_fn=serving_input_receiver_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predictions', 'serving_default']
INFO:tensorflow:Restoring parameters from train/train_gs_2/model.ckpt-24676
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b"trained/trained_0/temp-b'1532617597'/saved_model.pb"


In [15]:
print(mean_image, std_image)

103.659775 52.517044
